In [1]:


# 亚马逊中国“少儿销售排行榜”，前100位图书
# 书名，图片，价格，评论数，评分，页数，读者对象，开本，商品尺寸，商品重量，图书销量排名

#Step 1: 整理所有书籍的url
#Step 2：逐个访问，抓取目标数据，存储数据
#Step 3：生成csv文件

In [3]:
from lxml import html
import csv
import os
import requests
from time import sleep
from random import randint

def parse(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'
    }
    
    try:
        # Retrying for failed requests
        for i in range(20):
            # Generating random delays
            sleep(randint(1,3))
            # Adding verify=False to avold ssl related issues
            response = requests.get(url, headers=headers, verify=False)

            if response.status_code == 200: 
                doc = html.fromstring(response.content)
                XPATH_NAME = '//h1[@id="title"]//text()'
                XPATH_SALE_PRICE = '//span[@class="a-size-medium a-color-price inlineBlock-display offer-price a-text-normal price3P"]//text()'
                #'//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
                XPATH_COMMENT_NUMBER = '//span[@id="acrCustomerReviewText"]//text()'
                #'//span[@contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
                XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
                XPATH_PAGES = '//div[@class="content"]//text()'
                #'//div[@id="availability"]//text()'
                XPATH_READERS = '//div[@class="content"]//text()'
                XPATH_FORMAT = '//div[@class="content"]//text()'
                XSALES_RANK = '//li[@id="SalesRank"]//text()'
                XSIZE = '//div[@class="content"]//text()'
                XWEIGHT = '//div[@class="content"]//text()'
                
                
                
                RAW_NAME = doc.xpath(XPATH_NAME)
                RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
                #print(RAW_SALE_PRICE)
                RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
                #print (RAW_CATEGORY)
                RAW_COMMENT_NUMBER = doc.xpath(XPATH_COMMENT_NUMBER)
                #print (RAW_COMMENT_NUMBER)
                RAW_PAGES = doc.xpath(XPATH_PAGES)
                #print (RAW_PAGES.index(' 254页'))
                RAW_READERS = doc.xpath(XPATH_READERS)
                #print(RAW_READERS)
                #print (RAW_READERS.index(' 7-10岁'))
                RAW_FORMAT = doc.xpath(XPATH_FORMAT)
                #print(RAW_FORMAT.index(' 32'))
                RAW_SALES_RANK = doc.xpath(XSALES_RANK)
                #print (RAW_SALES_RANK)
                RAW_SIZE = doc.xpath(XSIZE)
                RAW_WEIGHT = doc.xpath(XWEIGHT)

                NAME = ''.join(''.join(RAW_NAME).split()[0]) if RAW_NAME else None
                # NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
                SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
                CATEGORY = ''.join(''.join(RAW_CATEGORY).split()[2]) if RAW_CATEGORY else None
                # ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
                COMMENT_NUMBER = ''.join(RAW_COMMENT_NUMBER).strip() if RAW_COMMENT_NUMBER else None
                #nPAGES = ''.join(''.join(RAW_PAGES).split()) if RAW_PAGES else None 
                for i in RAW_PAGES:
                    if '页' in i:
                        PAGES = RAW_PAGES[RAW_PAGES.index(i)].strip()
                        break
                #PAGES = nPAGES[nPAGES.index('页')-3:nPAGES.index('页')+1]
                #''.join(RAW_PAGES).strip() if RAW_PAGES else None
                # RAW_PAGES[224].strip() if RAW_PAGES else None
                #''.join(RAW_PAGES).strip() if RAW_PAGES else None
                for i in RAW_READERS:
                    if '岁' in i:
                        READERS = RAW_READERS[RAW_READERS.index(i)].strip()
                        break                
                #nREADERS = ''.join(''.join(RAW_READERS).split()) if RAW_READERS else None 
                #READERS = nREADERS[nREADERS.index('岁')-5:nREADERS.index('岁')+1]
                #RAW_READERS[261].strip() if RAW_READERS else None
                for i in RAW_FORMAT:
                    if '开本' in i:
                        FORMAT = RAW_FORMAT[RAW_FORMAT.index(i)+1].strip()
                        break
                #nFORMAT = ''.join(''.join(RAW_FORMAT).split()) if RAW_FORMAT else None
                #FORMAT = nFORMAT[nFORMAT.index('开本:')+3:nFORMAT.index('开本:')+5]
                #RAW_FORMAT[265].strip() if RAW_FORMAT else None
                SALES_RANK = ''.join(''.join(RAW_SALES_RANK).split()[0:2]).strip() if RAW_SALE_PRICE else None
                for i in RAW_SIZE:
                    if '尺寸' in i:
                        SIZE = RAW_SIZE[RAW_SIZE.index(i)+1].strip()
                        break                
                #nSIZE = ''.join(''.join(RAW_FORMAT).split()) if RAW_FORMAT else None
                #SIZE = nSIZE[nSIZE.index('尺寸'):nSIZE.index('尺寸')+16]
                #nWEIGHT = ''.join(''.join(RAW_WEIGHT).split()) if RAW_WEIGHT else None
                #WEIGHT = nWEIGHT[nWEIGHT.index('重量'):nWEIGHT.index('重量')+7]
                for i in RAW_WEIGHT:
                    if '重量' in i:
                        WEIGHT = RAW_WEIGHT[RAW_WEIGHT.index(i)+1].strip()
                        break  

                #if not ORIGINAL_PRICE:
                #    ORIGINAL_PRICE = SALE_PRICE
                # retrying in case of captcha
                if not NAME:
                    raise ValueError('captcha')

                data = {
                    'NAME': NAME,
                    'SALE_PRICE': SALE_PRICE,
                    'CATEGORY': CATEGORY,
                    'COMMENT_NUMBER': COMMENT_NUMBER,
                    'PAGES': PAGES,
                    'READERS':READERS,
                    'FORMAT':FORMAT,
                    'SALES_RANK':SALES_RANK,
                    'SIZE':SIZE,
                    'WEIGHT':WEIGHT,
                    'URL':url
                }
                print (data)
                return data
            
            elif response.status_code==404:
                break

    except Exception as e:
        print (e)

def ReadAsin():
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    AsinList =['/dp/B075KZKKTC/ref=zg_bs_658739051_1/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B079ZSFHD6/ref=zg_bs_658739051_2/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B00OLCRNHG/ref=zg_bs_658739051_3/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01CXUSAFG/ref=zg_bs_658739051_4/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B013FOXIWO/ref=zg_bs_658739051_5/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01J0OVBQI/ref=zg_bs_658739051_6/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01DXT2IXQ/ref=zg_bs_658739051_7/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B014ZZ6V44/ref=zg_bs_658739051_8/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B06Y6531DF/ref=zg_bs_658739051_9/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B075QDTY6C/ref=zg_bs_658739051_10/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01N9HR8X7/ref=zg_bs_658739051_11/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B0195V8WN6/ref=zg_bs_658739051_12/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B06XYJCYCF/ref=zg_bs_658739051_13/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01ANMKYBO/ref=zg_bs_658739051_14/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B06XXJSVXJ/ref=zg_bs_658739051_15/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01N2RJ3Z5/ref=zg_bs_658739051_16/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B00GHAST7I/ref=zg_bs_658739051_17/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B017U0LE1Q/ref=zg_bs_658739051_18/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B012VC7GS8/ref=zg_bs_658739051_19/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B01F6V0P6G/ref=zg_bs_658739051_20/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B079ZSFHD6/ref=zg_bs_tab_pd_bsnr_1/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B078F9KHSQ/ref=zg_bs_tab_pd_bsnr_2/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B078851G31/ref=zg_bs_tab_pd_bsnr_3/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N', '/dp/B003MNI4EQ/ref=zg_bs_660550051_1/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B07B14J2KK/ref=zg_bs_660550051_2/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B076VY8Z1H/ref=zg_bs_660550051_3/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B01MPXYVEJ/ref=zg_bs_660550051_4/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B004PLMLZW/ref=zg_bs_660550051_5/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B072RBZ6QV/ref=zg_bs_660550051_6/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B0089790OI/ref=zg_bs_660550051_7/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B003MNI4E6/ref=zg_bs_660550051_8/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B0089AGI0O/ref=zg_bs_660550051_9/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B06XW4FN18/ref=zg_bs_660550051_10/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B00CXKGMA2/ref=zg_bs_660550051_11/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B072BWBVYD/ref=zg_bs_660550051_12/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B01AUED0DE/ref=zg_bs_660550051_13/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B0076JD6UE/ref=zg_bs_660550051_14/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B01J0OVBQI/ref=zg_bs_660550051_15/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B00SMA88JK/ref=zg_bs_660550051_16/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B00EI4IFHY/ref=zg_bs_660550051_17/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B0087NSVN0/ref=zg_bs_660550051_18/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B0749KSDQ9/ref=zg_bs_660550051_19/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B018HVXQEU/ref=zg_bs_660550051_20/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B07B5Y8RW2/ref=zg_bs_tab_pd_bsnr_1/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B07CF6WGK1/ref=zg_bs_tab_pd_bsnr_2/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B07BF854RS/ref=zg_bs_tab_pd_bsnr_3/461-9588389-6100036?_encoding=UTF8&psc=1&refRID=E3PGEHAQY8N6MFQG07HZ', '/dp/B01E1G75BK/ref=zg_bs_660552051_1/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B071VSDKCF/ref=zg_bs_660552051_2/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B003MNI4EQ/ref=zg_bs_660552051_3/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B01N7LX5WC/ref=zg_bs_660552051_4/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B00F871Y4G/ref=zg_bs_660552051_5/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B071KVF8HL/ref=zg_bs_660552051_6/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B00B5NSXWI/ref=zg_bs_660552051_7/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B077ZSH7TH/ref=zg_bs_660552051_8/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07B16NSS1/ref=zg_bs_660552051_9/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07C9D8T3V/ref=zg_bs_660552051_10/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B00UJF093O/ref=zg_bs_660552051_11/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B007E85RKO/ref=zg_bs_660552051_12/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B01MZ01TU8/ref=zg_bs_660552051_13/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B074WTYF2Q/ref=zg_bs_660552051_14/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B06X6FRXRF/ref=zg_bs_660552051_15/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B004UD7R4A/ref=zg_bs_660552051_16/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07CPK4SF4/ref=zg_bs_660552051_17/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B004UD7R1I/ref=zg_bs_660552051_18/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07BF681DP/ref=zg_bs_660552051_19/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B06X6FTDZX/ref=zg_bs_660552051_20/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07B16NSS1/ref=zg_bs_tab_pd_bsnr_1/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07C9D8T3V/ref=zg_bs_tab_pd_bsnr_2/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B07CPK4SF4/ref=zg_bs_tab_pd_bsnr_3/457-8110344-6386461?_encoding=UTF8&psc=1&refRID=FVXPZ376G1DK17R7RX65', '/dp/B0083DP0CY/ref=zg_bs_658736051_1/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B01E1G75BK/ref=zg_bs_658736051_2/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B07CPF7N9S/ref=zg_bs_658736051_3/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B075KZKKTC/ref=zg_bs_658736051_4/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B0719JBX79/ref=zg_bs_658736051_5/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B009HIT5B6/ref=zg_bs_658736051_6/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B01ARP0NE0/ref=zg_bs_658736051_7/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B076DQZ3SY/ref=zg_bs_658736051_8/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00SMLR1S8/ref=zg_bs_658736051_9/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00RRDKWN8/ref=zg_bs_658736051_10/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00OLCRNHG/ref=zg_bs_658736051_11/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B01ASLP186/ref=zg_bs_658736051_12/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00KY4UWFK/ref=zg_bs_658736051_13/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B06XJX8CV2/ref=zg_bs_658736051_14/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00K9YU24Q/ref=zg_bs_658736051_15/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B0788WXTF9/ref=zg_bs_658736051_16/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B076VY8YZX/ref=zg_bs_658736051_17/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B00LM64IEK/ref=zg_bs_658736051_18/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B06Y3PQKCQ/ref=zg_bs_658736051_19/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B01MFDEMM1/ref=zg_bs_658736051_20/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B07CPF7N9S/ref=zg_bs_tab_pd_bsnr_1/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B07C54112N/ref=zg_bs_tab_pd_bsnr_2/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B07CPF7ML9/ref=zg_bs_tab_pd_bsnr_3/459-4403857-4160100?_encoding=UTF8&psc=1&refRID=EGS0D3K3T3454SC8SQR8', '/dp/B0788XVV4G/ref=zg_bs_660501051_1/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B01M2C43IQ/ref=zg_bs_660501051_2/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B071KVF8HL/ref=zg_bs_660501051_3/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B0035RX3S6/ref=zg_bs_660501051_4/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B00OLCRNHG/ref=zg_bs_660501051_5/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B01K49LAMI/ref=zg_bs_660501051_6/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B008HOHS4W/ref=zg_bs_660501051_7/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B0119FN2PO/ref=zg_bs_660501051_8/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B00LM64IEK/ref=zg_bs_660501051_9/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B06Y3PQKCQ/ref=zg_bs_660501051_10/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B01MFDEMM1/ref=zg_bs_660501051_11/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B00NBGXEHQ/ref=zg_bs_660501051_12/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B00E92VH0C/ref=zg_bs_660501051_13/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B06Y424PJM/ref=zg_bs_660501051_14/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B06Y248G48/ref=zg_bs_660501051_15/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B01J0OVBQI/ref=zg_bs_660501051_16/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B06Y1S4ZGL/ref=zg_bs_660501051_17/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B012C1S4X4/ref=zg_bs_660501051_18/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B005AQCBH4/ref=zg_bs_660501051_19/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B016IW62GO/ref=zg_bs_660501051_20/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B07BFB7L7S/ref=zg_bs_tab_pd_bsnr_1/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B07B5WB9YJ/ref=zg_bs_tab_pd_bsnr_2/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B07BN7Q1CG/ref=zg_bs_tab_pd_bsnr_3/457-3975062-2192066?_encoding=UTF8&psc=1&refRID=PGPNC1BFKRXSBCSJCRTF', '/dp/B01G8WDZ7I/ref=zg_bs_1978363051_1/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B001O09Z92/ref=zg_bs_1978363051_2/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B0791T9X6K/ref=zg_bs_1978363051_3/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B074R57XT1/ref=zg_bs_1978363051_4/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B00MENOF16/ref=zg_bs_1978363051_5/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B01M3QY7DQ/ref=zg_bs_1978363051_6/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B01MT3E4QM/ref=zg_bs_1978363051_7/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B019Z24Q5I/ref=zg_bs_1978363051_8/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07923NK23/ref=zg_bs_1978363051_9/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B00JEXDIMQ/ref=zg_bs_1978363051_10/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B00K9Z5JCA/ref=zg_bs_1978363051_11/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B00K9Z5JCU/ref=zg_bs_1978363051_12/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07D7SX5Q7/ref=zg_bs_1978363051_13/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B06ZYXT97Q/ref=zg_bs_1978363051_14/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B071RV2KGV/ref=zg_bs_1978363051_15/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B01BSZD2NW/ref=zg_bs_1978363051_16/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07D4ZSW77/ref=zg_bs_1978363051_17/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B01CY8H5B2/ref=zg_bs_1978363051_18/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B0192OZV2G/ref=zg_bs_1978363051_19/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B01J2XHJXG/ref=zg_bs_1978363051_20/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07D4ZSW77/ref=zg_bs_tab_pd_bsnr_1/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07C5K7SVD/ref=zg_bs_tab_pd_bsnr_2/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B07D4ZPJXP/ref=zg_bs_tab_pd_bsnr_3/459-0691319-2802250?_encoding=UTF8&psc=1&refRID=472YQZFZ52F5EQ516KKH', '/dp/B0795R113R/ref=zg_bs_660536051_1/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B076MG4TR5/ref=zg_bs_660536051_2/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B010PJ79N6/ref=zg_bs_660536051_3/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B013SKX9FQ/ref=zg_bs_660536051_4/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01MD1JLR3/ref=zg_bs_660536051_5/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B07CXDB7CP/ref=zg_bs_660536051_6/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01E6YAT4C/ref=zg_bs_660536051_7/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B012ZYORY8/ref=zg_bs_660536051_8/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B06X98973B/ref=zg_bs_660536051_9/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B0050GT4WE/ref=zg_bs_660536051_10/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01BR4GM9A/ref=zg_bs_660536051_11/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B003WMBTYO/ref=zg_bs_660536051_12/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B009EOKW0G/ref=zg_bs_660536051_13/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B0773C4G7K/ref=zg_bs_660536051_14/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B008AFF36I/ref=zg_bs_660536051_15/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01MQFHZFT/ref=zg_bs_660536051_16/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B0725PCH9Q/ref=zg_bs_660536051_17/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B00PSU2HVC/ref=zg_bs_660536051_18/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B004QQ5F4A/ref=zg_bs_660536051_19/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01D4LIJXG/ref=zg_bs_660536051_20/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B07CXDB7CP/ref=zg_bs_tab_pd_bsnr_1/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B079JPNZ9Z/ref=zg_bs_tab_pd_bsnr_2/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B07B12HXL6/ref=zg_bs_tab_pd_bsnr_3/459-2731177-5597909?_encoding=UTF8&psc=1&refRID=SKPNJ861RG422ZRH45V1', '/dp/B01M8KPLAR/ref=zg_bs_660532051_1/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00358JRG2/ref=zg_bs_660532051_2/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B077RJHV9V/ref=zg_bs_660532051_3/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B07BXTP119/ref=zg_bs_660532051_4/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00H4SZDA8/ref=zg_bs_660532051_5/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B008LQ96I2/ref=zg_bs_660532051_6/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00CK2UG1O/ref=zg_bs_660532051_7/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00CR6LTE6/ref=zg_bs_660532051_8/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00CAJXTKW/ref=zg_bs_660532051_9/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B01MS2L2JN/ref=zg_bs_660532051_10/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00JO9LYVC/ref=zg_bs_660532051_11/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B0197R4K0M/ref=zg_bs_660532051_12/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00UNFEKEY/ref=zg_bs_660532051_13/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B016M9ON2I/ref=zg_bs_660532051_14/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B079FNTYX1/ref=zg_bs_660532051_15/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B010N19NVC/ref=zg_bs_660532051_16/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B005HSLQ4E/ref=zg_bs_660532051_17/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B003SQH5BA/ref=zg_bs_660532051_18/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00KHLGJR0/ref=zg_bs_660532051_19/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B00H4SZ3IA/ref=zg_bs_660532051_20/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B07CLRY87R/ref=zg_bs_tab_pd_bsnr_1/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B079J8RFL4/ref=zg_bs_tab_pd_bsnr_2/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B079JGMNPY/ref=zg_bs_tab_pd_bsnr_3/459-6157728-6146345?_encoding=UTF8&psc=1&refRID=7RJ8VFBX12R4WVV650T6', '/dp/B071S6ZX3Z/ref=zg_bs_2151668051_1/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01MA5TMAZ/ref=zg_bs_2151668051_2/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B003MNI4EQ/ref=zg_bs_2151668051_3/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00DJ0F1LQ/ref=zg_bs_2151668051_4/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01KX4077S/ref=zg_bs_2151668051_5/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00NDF5HNE/ref=zg_bs_2151668051_6/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00TMX2STC/ref=zg_bs_2151668051_7/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00ZBUC4BM/ref=zg_bs_2151668051_8/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01MZ01TU8/ref=zg_bs_2151668051_9/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B071XTCBWH/ref=zg_bs_2151668051_10/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B075R1VTX9/ref=zg_bs_2151668051_11/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B018FXN8IO/ref=zg_bs_2151668051_12/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B074R57XT1/ref=zg_bs_2151668051_13/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00NDF5I06/ref=zg_bs_2151668051_14/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B00K0B18EQ/ref=zg_bs_2151668051_15/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01MQW6Z7T/ref=zg_bs_2151668051_16/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B003MNI4E6/ref=zg_bs_2151668051_17/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01M3QY7DQ/ref=zg_bs_2151668051_18/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B01N05W9XL/ref=zg_bs_2151668051_19/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B078M3XGM7/ref=zg_bs_2151668051_20/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B07BF2S4N2/ref=zg_bs_tab_pd_bsnr_1/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B07D2TMXVP/ref=zg_bs_tab_pd_bsnr_2/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B07CXDB7CP/ref=zg_bs_tab_pd_bsnr_3/462-9140817-8023425?_encoding=UTF8&psc=1&refRID=BVS3EBMBSE51B8HA3Z5F', '/dp/B07BFBT318/ref=zg_bs_658737051_1/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00OPNOT78/ref=zg_bs_658737051_2/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B074JS9Q8P/ref=zg_bs_658737051_3/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B003OUW7CC/ref=zg_bs_658737051_4/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B071DFQB5P/ref=zg_bs_658737051_5/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00HYEFS66/ref=zg_bs_658737051_6/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B006OG3SNU/ref=zg_bs_658737051_7/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B006MA7AKU/ref=zg_bs_658737051_8/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B0757G2PKR/ref=zg_bs_658737051_9/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00K8AJAMG/ref=zg_bs_658737051_10/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B01N7A29W7/ref=zg_bs_658737051_11/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B01NANJEPL/ref=zg_bs_658737051_12/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00LZG4WZ2/ref=zg_bs_658737051_13/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00HIYI6QQ/ref=zg_bs_658737051_14/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B0725PBXDN/ref=zg_bs_658737051_15/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B07D4ZWCLF/ref=zg_bs_658737051_16/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00HLC0EJW/ref=zg_bs_658737051_17/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B01N68X0U0/ref=zg_bs_658737051_18/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B06XR9X3QY/ref=zg_bs_658737051_19/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B00K6404XI/ref=zg_bs_658737051_20/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B07BFBT318/ref=zg_bs_tab_pd_bsnr_1/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B07D4ZWCLF/ref=zg_bs_tab_pd_bsnr_2/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B07BF3T9MQ/ref=zg_bs_tab_pd_bsnr_3/459-5508843-7778525?_encoding=UTF8&psc=1&refRID=3W6XGRD5AR1YBTPYVC9R', '/dp/B0083DP0CY/ref=zg_bs_658738051_1/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07BFBT318/ref=zg_bs_658738051_2/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00XJ00NPK/ref=zg_bs_658738051_3/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00CY8HN3S/ref=zg_bs_658738051_4/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B0072T4OV8/ref=zg_bs_658738051_5/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B071F4PWJZ/ref=zg_bs_658738051_6/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B01E599034/ref=zg_bs_658738051_7/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00KKISDB0/ref=zg_bs_658738051_8/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00QLEU0YA/ref=zg_bs_658738051_9/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B071DFQB5P/ref=zg_bs_658738051_10/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B01G8WDZ7I/ref=zg_bs_658738051_11/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07BZBW8KN/ref=zg_bs_658738051_12/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B06XT8N3YB/ref=zg_bs_658738051_13/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00COW4FM6/ref=zg_bs_658738051_14/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B071S6ZX3Z/ref=zg_bs_658738051_15/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B01MA5TMAZ/ref=zg_bs_658738051_16/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B005EFU33K/ref=zg_bs_658738051_17/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00KYN7AY2/ref=zg_bs_658738051_18/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07895XRKN/ref=zg_bs_658738051_19/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B008HXEOW2/ref=zg_bs_658738051_20/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07BFBT318/ref=zg_bs_tab_pd_bsnr_1/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07895XRKN/ref=zg_bs_tab_pd_bsnr_2/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B07CBLS7BL/ref=zg_bs_tab_pd_bsnr_3/459-2908141-0482513?_encoding=UTF8&psc=1&refRID=NVEVRTT2WXAB5WA52K4Q', '/dp/B00XJ00NPK/ref=zg_bs_658735051_1/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B00ANFMU8W/ref=zg_bs_658735051_2/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07BF681K4/ref=zg_bs_658735051_3/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B0192NTFDI/ref=zg_bs_658735051_4/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B004INGC7U/ref=zg_bs_658735051_5/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B071JW9SVF/ref=zg_bs_658735051_6/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B00LTOJVRO/ref=zg_bs_658735051_7/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B06XT8N3YB/ref=zg_bs_658735051_8/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B003YJFNME/ref=zg_bs_658735051_9/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B0774PMDN8/ref=zg_bs_658735051_10/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07895XRKN/ref=zg_bs_658735051_11/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B01BY6E98W/ref=zg_bs_658735051_12/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07CBLS7BL/ref=zg_bs_658735051_13/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B079ZSVNC3/ref=zg_bs_658735051_14/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B0757G2PKR/ref=zg_bs_658735051_15/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B001N6R68O/ref=zg_bs_658735051_16/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B00OUNPC1A/ref=zg_bs_658735051_17/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B075JDMXJN/ref=zg_bs_658735051_18/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B0774CLCML/ref=zg_bs_658735051_19/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B01ARP0NE0/ref=zg_bs_658735051_20/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07BF681K4/ref=zg_bs_tab_pd_bsnr_1/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07895XRKN/ref=zg_bs_tab_pd_bsnr_2/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B07CBLS7BL/ref=zg_bs_tab_pd_bsnr_3/461-2707016-5253049?_encoding=UTF8&psc=1&refRID=ZZEQW2CZMFP05J1Q48YT', '/dp/B0083DP0CY/ref=zg_bs_658734051_1/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00OPNOT78/ref=zg_bs_658734051_2/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B071DFQB5P/ref=zg_bs_658734051_3/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B01CHBW1SI/ref=zg_bs_658734051_4/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B01MA5TMAZ/ref=zg_bs_658734051_5/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B0774PMDN8/ref=zg_bs_658734051_6/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B009HIT5B6/ref=zg_bs_658734051_7/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00R2NWUSI/ref=zg_bs_658734051_8/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B07CPDKQ93/ref=zg_bs_658734051_9/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00F871Y4G/ref=zg_bs_658734051_10/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00NDF5HNE/ref=zg_bs_658734051_11/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B0011A8RKI/ref=zg_bs_658734051_12/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00M3S8450/ref=zg_bs_658734051_13/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00RRDKWN8/ref=zg_bs_658734051_14/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B0035RX3S6/ref=zg_bs_658734051_15/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B00XJ00G0W/ref=zg_bs_658734051_16/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B01ABGNFUO/ref=zg_bs_658734051_17/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B007E85RKO/ref=zg_bs_658734051_18/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B01ASLP186/ref=zg_bs_658734051_19/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B01MZ01TU8/ref=zg_bs_658734051_20/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B07CPDKQ93/ref=zg_bs_tab_pd_bsnr_1/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B07BSLQGJH/ref=zg_bs_tab_pd_bsnr_2/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT', '/dp/B07D2TMXVP/ref=zg_bs_tab_pd_bsnr_3/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT']
    extracted_data = []

    for i in AsinList:
        url = "https://www.amazon.cn" + i
        print ("Processing: " + url)
        # Calling the parser
        parsed_data = parse(url)
        if parsed_data:
            extracted_data.append(parsed_data)

    # Writing scraped data to csv file
    with open('scraped_data.csv', 'w') as csvfile:
        fieldnames = ['NAME','SALE_PRICE','CATEGORY','COMMENT_NUMBER','PAGES','READERS','FORMAT','SALES_RANK','SIZE','WEIGHT','URL']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)
        writer.writeheader()    
        for data in extracted_data:
            writer.writerow(data)

if __name__ == "__main__":
    ReadAsin()

Processing: https://www.amazon.cn/dp/B075KZKKTC/ref=zg_bs_658739051_1/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'NAME': '超好玩的立体翻翻书:车辆动起来+船舶动起来(套装共2册)', 'SALE_PRICE': '￥300.00', 'CATEGORY': '智力游戏', 'COMMENT_NUMBER': '24 条商品评论', 'PAGES': '50页', 'READERS': '3-6岁', 'FORMAT': '8', 'SALES_RANK': '亚马逊热销商品排名:图书商品里排第386名', 'SIZE': '34 x 26.9 x 6.9 cm', 'WEIGHT': '3.4 Kg', 'URL': 'https://www.amazon.cn/dp/B075KZKKTC/ref=zg_bs_658739051_1/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N'}
Processing: https://www.amazon.cn/dp/B079ZSFHD6/ref=zg_bs_658739051_2/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'NAME': '0-3岁触摸立体翻翻书:和小鸡球球一起玩(套装共6册)', 'SALE_PRICE': '￥116.00', 'CATEGORY': '少儿绘本与漫画', 'COMMENT_NUMBER': '5 条商品评论', 'PAGES': '144页', 'READERS': '0-2岁', 'FORMAT': '48', 'SALES_RANK': '亚马逊热销商品排名:图书商品里排第585名', 'SIZE': '14.4 x 14.4 x 7.8 cm', 'WEIGHT': '780 g', 'URL': 'https://www.amazon.cn/dp/B079ZSFHD6/ref=zg_bs_658739051_2/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N'}
Processing: https://www.amazon.cn/dp/B00OLCRNHG/ref=zg_bs_658739051_3/457-0188065-3712778?_encoding=UTF8&psc=1&refRID=AY57NHQXZQX1T8ZS6V4N


...

{'NAME': '创意大师拉拉翻翻大惊喜系列(套装共7册)', 'SALE_PRICE': '￥256.16', 'CATEGORY': '幼儿启蒙', 'COMMENT_NUMBER': None, 'PAGES': '186页', 'READERS': '3-6岁', 'FORMAT': '24', 'SALES_RANK': '亚马逊热销商品排名:图书商品里排第562名', 'SIZE': '19 x 18.2 x 11.2 cm', 'WEIGHT': '2.02 Kg', 'URL': 'https://www.amazon.cn/dp/B07CPDKQ93/ref=zg_bs_tab_pd_bsnr_1/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT'}
Processing: https://www.amazon.cn/dp/B07BSLQGJH/ref=zg_bs_tab_pd_bsnr_2/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{'NAME': '信谊·宝宝起步走:抱抱', 'SALE_PRICE': '￥30.20', 'CATEGORY': '幼儿启蒙', 'COMMENT_NUMBER': '593 条商品评论', 'PAGES': '40页', 'READERS': '0-2岁', 'FORMAT': '12', 'SALES_RANK': '亚马逊热销商品排名:图书商品里排第1,902名', 'SIZE': '27.6 x 23.4 x 0.8 cm', 'WEIGHT': '422 g', 'URL': 'https://www.amazon.cn/dp/B07BSLQGJH/ref=zg_bs_tab_pd_bsnr_2/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT'}
Processing: https://www.amazon.cn/dp/B07D2TMXVP/ref=zg_bs_tab_pd_bsnr_3/459-0685494-7892567?_encoding=UTF8&psc=1&refRID=F58KFT55EYC6ZNE008QT


/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


local variable 'READERS' referenced before assignment
